In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import altair as alt

In [2]:
from ecostyles import EcoStyles
# Create styles instance
styles = EcoStyles()
# Register and enable a theme
styles.register_and_enable_theme(theme_name="article")  # or "article"

In [159]:
styles.eco_colours

{'red': '#e6224b',
 'blue-light': '#179fdb',
 'blue-dark': '#122b39',
 'yellow': '#f4c245',
 'orange': '#eb5c2e',
 'turquoise': '#36b7b4'}

In [3]:
hmrc = pd.read_csv('/Users/sambickel-barlow/RADataHub/RADataHub/ChartOfTheDay/UK tax/HM Revenue and Customs Receipts.csv')

In [4]:
gdp = pd.read_csv('/Users/sambickel-barlow/RADataHub/RADataHub/ChartOfTheDay/UK tax/UK_GDP_FY.csv')

In [5]:
sdlt = hmrc[['Table 2 HMRC Receipts by financial year','Stamp Duty Land Tax', 'Total HMRC Receipts']].rename(columns={'Table 2 HMRC Receipts by financial year':'FY','Stamp Duty Land Tax':'SDLT'})
inht = hmrc[['Table 2 HMRC Receipts by financial year','Inheritance Tax', 'Total HMRC Receipts']].rename(columns={'Table 2 HMRC Receipts by financial year':'FY','Inheritance Tax':'IHT'})
gdp_fy = gdp[['Financial year', 'Cash £ million\nNon-Season-ally Adjusted']].rename(columns={'Financial year':'FY','Cash £ million\nNon-Season-ally Adjusted':'GDP'})

In [6]:
sdlt['FY'] = 'FY' + sdlt['FY'].str.split('to 20').str[1]
inht['FY'] = 'FY' + inht['FY'].str.split('to 20').str[1]
gdp_fy['FY'] = 'FY' + gdp_fy['FY'].str.split('-').str[1]

In [7]:
sdlt_gdp = sdlt.merge(gdp_fy, how='left', on='FY')
inht_gdp = inht.merge(gdp_fy, how='left', on='FY')

In [8]:
sdlt_gdp['GDP'] = sdlt_gdp['GDP'].str.replace(',','').astype(float)
sdlt_gdp['SDLT'] = sdlt_gdp['SDLT'].str.replace(',','').astype(float)
sdlt_gdp['Total HMRC Receipts'] = sdlt_gdp['Total HMRC Receipts'].str.replace(',','').astype(float)
sdlt_gdp['SDLT as % of GDP'] = sdlt_gdp['SDLT'] / sdlt_gdp['GDP']
sdlt_gdp['SDLT as % of Tax Receipts'] = sdlt_gdp['SDLT'] / sdlt_gdp['Total HMRC Receipts']

inht_gdp['GDP'] = inht_gdp['GDP'].str.replace(',','').astype(float)
inht_gdp['IHT'] = inht_gdp['IHT'].str.replace(',','').astype(float)
inht_gdp['Total HMRC Receipts'] = inht_gdp['Total HMRC Receipts'].str.replace(',','').astype(float)
inht_gdp['IHT as % of GDP'] = inht_gdp['IHT'] / inht_gdp['GDP']
inht_gdp['IHT as % of Tax Receipts'] = inht_gdp['IHT'] / inht_gdp['Total HMRC Receipts']

In [9]:
# Create base chart
base = alt.Chart(sdlt_gdp)

# SDLT bars (left y-axis)
bars = base.mark_bar(
    opacity=0.7,
    color='#1f77b4'
).encode(
    x=alt.X('FY:O', title=None, axis=alt.Axis(labelFontSize=12.5)),
    y=alt.Y('SDLT:Q', 
                title='SDLT receipts', 
                scale=alt.Scale(zero=True), 
                axis=alt.Axis(titleX=-76, titleFontSize=12.5, labelFontSize=12.5, labelExpr='"£" + datum.value / 1000 + " bn"')),
    #tooltip=[
    #    alt.Tooltip('FY:O', title='Financial Year'),
    #    alt.Tooltip('SDLT:Q', title='SDLT Revenue', format='£,.0f'),
        #alt.Tooltip('SDLT as % of GDP:Q', title='SDLT as % of GDP', format='.2%')
    #]
)

# SDLT as % of GDP line (right y-axis)
line = base.mark_line(
    color='#ff7f0e',
    strokeWidth=3,
    point=False
).encode(
    x='FY:O',
    y=alt.Y('SDLT as % of GDP:Q', 
            title='SDLT as % of GDP',
            scale=alt.Scale(zero=True),
            axis=alt.Axis(format=',.1%', titleX=0, titleFontSize=12.5, labelFontSize=12.5)),
    #tooltip=[
        #alt.Tooltip('FY:O', title='Financial Year'),
        #alt.Tooltip('SDLT:Q', title='SDLT Revenue', format='£,.0f'),
        #alt.Tooltip('SDLT as % of GDP:Q', title='SDLT as % of GDP', format='.2%')
    #]
)

footnote = alt.Chart(pd.DataFrame({'text': [
    "Sources: HMRC tax receipts and National Insurance contributions for the UK (annual bulletin)"
]})).mark_text(
    align='left',
    baseline='top',
    fontSize=15,
    fontStyle='italic',
    dy=225,
    x=-12.5
).encode(
    text='text'
)

# Combine charts with dual y-axes
chart = alt.layer(
    bars,
    line,
    footnote
).resolve_scale(
    y='independent'
).properties(
    width=600,
    height=400,
    title='Stamp Duty Land Tax (SDLT) Revenue and Share of GDP',
).configure_title(
    fontSize=20
)

chart

alt.LayerChart(...)

In [12]:
# Create base chart
base = alt.Chart(sdlt_gdp)

# SDLT bars (left y-axis)
bars = base.mark_bar(
    opacity=0.7,
).encode(
    x=alt.X('FY:O', title=None, axis=alt.Axis(labelFontSize=12.5)),
    y=alt.Y('SDLT:Q', 
                title='SDLT Receipts', 
                scale=alt.Scale(zero=True), 
                axis=alt.Axis(titleX=-81, titleY = -15, titleColor = '#179fdb', titleFontSize=14, labelFontSize=12.5, labelExpr='"£" + datum.value / 1000 + " bn"')),
    #tooltip=[
    #    alt.Tooltip('FY:O', title='Financial Year'),
    #    alt.Tooltip('SDLT:Q', title='SDLT Revenue', format='£,.0f'),
        #alt.Tooltip('SDLT as % of GDP:Q', title='SDLT as % of GDP', format='.2%')
    #]
)

# SDLT as % of GDP line (right y-axis)
line = base.mark_line(
    #color='#ff7f0e',
    strokeWidth=3,
    point=False
).encode(
    x='FY:O',
    y=alt.Y('SDLT as % of Tax Receipts:Q', 
            title=['SDLT % of', 'Tax Receipts'],
            scale=alt.Scale(zero=True),
            axis=alt.Axis(format=',.1%', titleX=0, titleY = -30, titleFontSize=14, labelFontSize=12.5, titleColor='#e6224b')),
    #tooltip=[
        #alt.Tooltip('FY:O', title='Financial Year'),
        #alt.Tooltip('SDLT:Q', title='SDLT Revenue', format='£,.0f'),
        #alt.Tooltip('SDLT as % of GDP:Q', title='SDLT as % of GDP', format='.2%')
    #]
)

footnote = alt.Chart(pd.DataFrame({'text': [
    "Sources: HMRC tax receipts and National Insurance contributions for the UK (annual bulletin)"
]})).mark_text(
    align='left',
    baseline='top',
    fontSize=13,
    fontStyle='italic',
    dy=225,
    x=-12.5
).encode(
    text='text'
)

# Combine charts with dual y-axes
chart = alt.layer(
    bars,
    line,
    footnote
).resolve_scale(
    y='independent'
).properties(
    width=600,
    height=400,
    title='Stamp Duty Land Tax (SDLT) Receipts',
).configure_title(
    fontSize=25,
    offset=10
)

chart

alt.LayerChart(...)

In [13]:
# Save to png
chart.save('SDLT.png', scale_factor=2)
chart.save('SDLT.json', scale_factor=2)

In [128]:
# Create base chart
base = alt.Chart(inht_gdp)

# IHT bars (left y-axis)
bars = base.mark_bar(
    opacity=0.7,
    color='#1f77b4'
).encode(
    x=alt.X('FY:O', title=None, axis=alt.Axis(labelFontSize=12.5)),
    y=alt.Y('IHT:Q', 
                title='IHT receipts', 
                scale=alt.Scale(zero=True), 
                axis=alt.Axis(titleX=-76, titleFontSize=12.5, labelFontSize=12.5, labelExpr='"£" + datum.value / 1000 + " bn"')),
    #tooltip=[
    #    alt.Tooltip('FY:O', title='Financial Year'),
    #    alt.Tooltip('SDLT:Q', title='SDLT Revenue', format='£,.0f'),
        #alt.Tooltip('SDLT as % of GDP:Q', title='SDLT as % of GDP', format='.2%')
    #]
)

# SDLT as % of GDP line (right y-axis)
line = base.mark_line(
    color='#ff7f0e',
    strokeWidth=3,
    point=False
).encode(
    x='FY:O',
    y=alt.Y('IHT as % of GDP:Q', 
            title='IHT as % of GDP',
            scale=alt.Scale(zero=True),
            axis=alt.Axis(format=',.2%', titleX=0, titleFontSize=12.5, labelFontSize=12.5)),
    #tooltip=[
        #alt.Tooltip('FY:O', title='Financial Year'),
        #alt.Tooltip('SDLT:Q', title='SDLT Revenue', format='£,.0f'),
        #alt.Tooltip('SDLT as % of GDP:Q', title='SDLT as % of GDP', format='.2%')
    #]
)

footnote = alt.Chart(pd.DataFrame({'text': [
    "Sources: HMRC tax receipts and National Insurance contributions for the UK (annual bulletin)"
]})).mark_text(
    align='left',
    baseline='top',
    fontSize=15,
    fontStyle='italic',
    dy=225,
    x=-12.5
).encode(
    text='text'
)

# Combine charts with dual y-axes
chart = alt.layer(
    bars,
    line,
    footnote
).resolve_scale(
    y='independent'
).properties(
    width=600,
    height=400,
    title='Inheritance Tax (IHT) Revenue and Share of GDP',
).configure_title(
    fontSize=20
)

chart

alt.LayerChart(...)

In [144]:
# Create base chart
base = alt.Chart(inht_gdp)

# IHT bars (left y-axis)
bars = base.mark_bar(
    opacity=0.7,
    color='#1f77b4'
).encode(
    x=alt.X('FY:O', title=None, axis=alt.Axis(labelFontSize=12.5)),
    y=alt.Y('IHT:Q', 
                title='IHT receipts', 
                scale=alt.Scale(zero=True), 
                axis=alt.Axis(titleX=-76, titleFontSize=12.5, labelFontSize=12.5, labelExpr='"£" + datum.value / 1000 + " bn"')),
    #tooltip=[
    #    alt.Tooltip('FY:O', title='Financial Year'),
    #    alt.Tooltip('SDLT:Q', title='SDLT Revenue', format='£,.0f'),
        #alt.Tooltip('SDLT as % of GDP:Q', title='SDLT as % of GDP', format='.2%')
    #]
)

# SDLT as % of GDP line (right y-axis)
line = base.mark_line(
    color='#ff7f0e',
    strokeWidth=3,
    point=False
).encode(
    x='FY:O',
    y=alt.Y('IHT as % of Total Tax Revenue:Q', 
            title='IHT as % of Total Tax Revenue',
            scale=alt.Scale(zero=True),
            axis=alt.Axis(format=',.1%', titleX=0, titleFontSize=12.5, labelFontSize=12.5)),
    #tooltip=[
        #alt.Tooltip('FY:O', title='Financial Year'),
        #alt.Tooltip('SDLT:Q', title='SDLT Revenue', format='£,.0f'),
        #alt.Tooltip('SDLT as % of GDP:Q', title='SDLT as % of GDP', format='.2%')
    #]
)

footnote = alt.Chart(pd.DataFrame({'text': [
    "Sources: HMRC tax receipts and National Insurance contributions for the UK (annual bulletin)"
]})).mark_text(
    align='left',
    baseline='top',
    fontSize=15,
    fontStyle='italic',
    dy=225,
    x=-12.5
).encode(
    text='text'
)

# Combine charts with dual y-axes
chart = alt.layer(
    bars,
    line,
    footnote
).resolve_scale(
    y='independent'
).properties(
    width=600,
    height=400,
    title='Inheritance Tax (IHT) and Share of Total Tax Receipts',
).configure_title(
    fontSize=20
)

chart

alt.LayerChart(...)

In [164]:
# Save to png
chart.save('SDLT.png', scale_factor=2)
chart.save('SDLT.json', scale_factor=2)